# 하나은행의 환율 조회부분 찾아서 selenium으로 열기
* 날짜 입력부분 찾아 날짜 넣기
* 조회 버튼 찾아 버튼 클릭하기
* 조회된 데이터를 pd.read_html을 이용해 dataframe으로 만들기
* 데이터 프레임에 조회 날짜 컬럼 만들어서 날짜 추가하기

In [120]:
def new_cols(df):
    new_cols =[]
    for cols in df.columns:
        if cols[0] == cols[1] == cols[0]:
            new_cols.append(cols[0].replace(" ","_"))
        else:
            new_cols.append(" ".join(cols).strip().replace(" ","_"))
    return new_cols

In [130]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
import time 
import pandas as pd
from io import StringIO
from exdb_io import to_ex_db

options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/135.0.0.0")
options.add_argument("lang=ko_KR")


driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
    )
driver.get("https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_01i.do#//HanaBank")

# 시작일 지정 (예: 2024년 1월 1일)
start_date = datetime(1995, 1, 3)

# 어제 날짜 구하기
yesterday = datetime.today() - timedelta(days=1)

# 날짜를 'YYYY-MM-DD' 형식으로 변환
start_str = start_date.strftime("%Y-%m-%d")
end_str = yesterday.strftime("%Y-%m-%d")

current_date = start_date

date_list = []
while current_date <= yesterday :
    if current_date.weekday() < 5:
        wait = WebDriverWait(driver, 10)
        date_element = wait.until(EC.presence_of_element_located((By.ID,"tmpInqStrDt")))
        date_element.clear()
        date_element.send_keys(f"{current_date}")
                                        
        btn = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".btnDefault.bg")))
        btn.click()
        
        date_list.append(current_date)                                                         
        df = pd.read_html(StringIO(driver.find_element(By.CSS_SELECTOR, ".tblBasic.leftNone").get_attribute("outerHTML")))[0]
        df['date'] =  '2024-04-05'
        new_columns = new_cols(df)
        df.columns = new_columns
        df = df[['date','통화','현찰_사실_때_환율','현찰_사실_때_Spread','현찰_파실_때_환율',
                 '현찰_파실_때_Spread','송금_보낼_때_보낼_때','송금_받을_때_받을_때',
                 '외화_수표_파실때','매매_기준율','환가_료율','미화_환산율']]
        to_ex_db(df)                                                         
                                                                 
    current_date += timedelta(days=1)
df

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=135.0.7049.43)
Stacktrace:
	GetHandleVerifier [0x00FB80E3+60707]
	GetHandleVerifier [0x00FB8124+60772]
	(No symbol) [0x00DE0683]
	(No symbol) [0x00DCFED0]
	(No symbol) [0x00DEDD3F]
	(No symbol) [0x00E53D7F]
	(No symbol) [0x00E6E129]
	(No symbol) [0x00E4CE46]
	(No symbol) [0x00E1C5D3]
	(No symbol) [0x00E1D424]
	GetHandleVerifier [0x011FBBC3+2435075]
	GetHandleVerifier [0x011F7163+2416035]
	GetHandleVerifier [0x0121350C+2531660]
	GetHandleVerifier [0x00FCF1B5+155125]
	GetHandleVerifier [0x00FD5B5D+182173]
	GetHandleVerifier [0x00FBF9B8+91640]
	GetHandleVerifier [0x00FBFB60+92064]
	GetHandleVerifier [0x00FAA620+4704]
	BaseThreadInitThunk [0x7526FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x773D82AE+286]
	RtlGetAppContainerNamedObjectPath [0x773D827E+238]


In [129]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import datetime
from io import StringIO
import time
from exdb_io import to_ex_db

options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/135.0.0.0")
options.add_argument("lang=ko_KR")


driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
    )

url = "https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_01i.do"
driver.get(url)

wait = WebDriverWait(driver, 10)

# 시작일과 끝일 입력해서 날짜 리스트 생성
start_date = datetime.date(2025, 4, 1)
end_date = datetime.date(2025, 4, 2)

date_list = []
while start_date <= end_date:
    # 평일인지 필터링
    if start_date.weekday() < 5:
        date_list.append(start_date)
    start_date += datetime.timedelta(days = 1)

for date in date_list:
    # 날짜 입력
    date_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#tmpInqStrDt")))
    date_input.clear()
    date_input.send_keys(str(date).replace("-", ""))
    date_input.send_keys(Keys.ENTER)

    # 조회버튼 클릭
    search_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#HANA_CONTENTS_DIV > div.btnBoxCenter > a")))
    search_button.click()
    time.sleep(2)
    df = pd.read_html(StringIO(driver.find_element(By.CSS_SELECTOR, ".tblBasic.leftNone").get_attribute('outerHTML')))[0]
    df['date'] = date
    new_columns = new_col(df)
    df.columns = new_columns
    df = df[['date', '통화', '현찰_사실_때_환율', '현찰_사실_때_Spread', '현찰_파실_때_환율', '현찰_파실_때_Spread',
       '송금_보낼_때_보낼_때', '송금_받을_때_받을_때', '외화_수표_파실때', '매매_기준율', '환가_료율',
       '미화_환산율']]
    to_ex_db(df)

NameError: name 'new_col' is not defined

In [9]:
driver.find_element(By.ID, "tmpInqStrDt")

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=135.0.7049.43)
Stacktrace:
	GetHandleVerifier [0x00FB80E3+60707]
	GetHandleVerifier [0x00FB8124+60772]
	(No symbol) [0x00DE0683]
	(No symbol) [0x00DCFED0]
	(No symbol) [0x00DEDD3F]
	(No symbol) [0x00E53D7F]
	(No symbol) [0x00E6E129]
	(No symbol) [0x00E4CE46]
	(No symbol) [0x00E1C5D3]
	(No symbol) [0x00E1D424]
	GetHandleVerifier [0x011FBBC3+2435075]
	GetHandleVerifier [0x011F7163+2416035]
	GetHandleVerifier [0x0121350C+2531660]
	GetHandleVerifier [0x00FCF1B5+155125]
	GetHandleVerifier [0x00FD5B5D+182173]
	GetHandleVerifier [0x00FBF9B8+91640]
	GetHandleVerifier [0x00FBFB60+92064]
	GetHandleVerifier [0x00FAA620+4704]
	BaseThreadInitThunk [0x7526FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x773D82AE+286]
	RtlGetAppContainerNamedObjectPath [0x773D827E+238]


In [7]:
driver.find_element(By.CSS_SELECTOR, ".btnDefault.bg").text

InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00FB80E3+60707]
	GetHandleVerifier [0x00FB8124+60772]
	(No symbol) [0x00DE04FE]
	(No symbol) [0x00E1B898]
	(No symbol) [0x00E4CF06]
	(No symbol) [0x00E489D5]
	(No symbol) [0x00E47F66]
	(No symbol) [0x00DB36E5]
	(No symbol) [0x00DB3C3E]
	(No symbol) [0x00DB40CD]
	GetHandleVerifier [0x011FBBC3+2435075]
	GetHandleVerifier [0x011F7163+2416035]
	GetHandleVerifier [0x0121350C+2531660]
	GetHandleVerifier [0x00FCF1B5+155125]
	GetHandleVerifier [0x00FD5B5D+182173]
	(No symbol) [0x00DB33B0]
	(No symbol) [0x00DB2BC3]
	GetHandleVerifier [0x0131D2AC+3620588]
	BaseThreadInitThunk [0x7526FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x773D82AE+286]
	RtlGetAppContainerNamedObjectPath [0x773D827E+238]


In [ ]:
tmpInqStrDt

In [14]:
driver.find_element(By.CSS_SELECTOR, "td.tc")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"td.tc"}
  (Session info: chrome=135.0.7049.43); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00FB80E3+60707]
	GetHandleVerifier [0x00FB8124+60772]
	(No symbol) [0x00DE0683]
	(No symbol) [0x00E28660]
	(No symbol) [0x00E289FB]
	(No symbol) [0x00E71022]
	(No symbol) [0x00E4D094]
	(No symbol) [0x00E6E824]
	(No symbol) [0x00E4CE46]
	(No symbol) [0x00E1C5D3]
	(No symbol) [0x00E1D424]
	GetHandleVerifier [0x011FBBC3+2435075]
	GetHandleVerifier [0x011F7163+2416035]
	GetHandleVerifier [0x0121350C+2531660]
	GetHandleVerifier [0x00FCF1B5+155125]
	GetHandleVerifier [0x00FD5B5D+182173]
	GetHandleVerifier [0x00FBF9B8+91640]
	GetHandleVerifier [0x00FBFB60+92064]
	GetHandleVerifier [0x00FAA620+4704]
	BaseThreadInitThunk [0x7526FCC9+25]
	RtlGetAppContainerNamed

In [83]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
import time 


options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/135.0.0.0")
options.add_argument("lang=ko_KR")


driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
    )
driver.get(f"https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_01i.do#//HanaBank")



In [65]:


for country in driver.find_elements(By.CSS_SELECTOR, "tbody"):
    #날짜 
    
    # 통화
    currency = country.find_element(By.CSS_SELECTOR, "td:nth-child(1)").text
    #현찰 살떄 환율
    cash_b = country.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text
    # 현찰 살때환율 Spread
    cash_b_s = country.find_element(By.CSS_SELECTOR, "td:nth-child(3)").text
    #현찰 팔떄 환율
    cash_s = country.find_element(By.CSS_SELECTOR, "td:nth-child(4)").text
    # 현찰 팔때환율 Spread
    cash_s_s = country.find_element(By.CSS_SELECTOR, "td:nth-child(5)").text
    #보낼때 환율
    send = country.find_element(By.CSS_SELECTOR, "td:nth-child(6)").text
    #받을떄 환율
    receive = country.find_element(By.CSS_SELECTOR, "td:nth-child(7)").text
    #외화수표
    check =country.find_element(By.CSS_SELECTOR, "td:nth-child(8)").text
    # 매매기준율
    per = country.find_element(By.CSS_SELECTOR, "td:nth-child(9)").text
    #환가요율
    tran = country.find_element(By.CSS_SELECTOR, "td:nth-child(10)").text
    #미화 환산률
    conversion_rate = country.find_element(By.CSS_SELECTOR, "td:nth-child(11)").text
    
    values = (currency, cash_b, cash_b_s, cash_s, cash_s_s, send, receive, check, per, tran, conversion_rate)
    columns = ("통화","현찰사실때","현찰사실때_Spread", "현찰파실때","현찰파실때_Spread","송금보낼때", "송금받을때", "외화수표파실때", "매매기준율", "환가료율", "미화환산율")
    df = pd.DataFrame([values],columns=columns )

df
    

,통화,현찰사실때,현찰사실때_Spread,현찰파실때,현찰파실때_Spread,송금보낼때,송금받을때,외화수표파실때,매매기준율,환가료율,미화환산율
0,미국 USD,"1,507.42",1.75,"1,455.58",1.75,"1,496.00","1,467.00","1,464.48","1,481.50",6.14181,1.0000


In [ ]:
#HANA_CONTENTS_DIV > div.btnBoxCenter > a

In [72]:
driver.find_elements(By.CSS_SELECTOR, "tbody").get_attribute("outerHTML")

AttributeError: 'list' object has no attribute 'get_attribute'

In [ ]:
driver.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text

In [56]:
driver.find_elements(By.CSS_SELECTOR, "thead")[0].text.replace("\n", "")

'통화 현찰 송금 외화수표파실때 매매기준율 환가료율 미화환산율사실 때 파실 때 보낼 때 받을 때환율 Spread 환율 Spread'

In [76]:
document.querySelector("#searchContentDiv > div.printdiv > table")

NameError: name 'document' is not defined

In [ ]:
df['date'] = []

# 날짜 생성하기
* 1995-01-03 ~ 어제까지(2025-04-08) 날짜 생성하기
* 평일만 날짜 생성
* 날짜 리스트 생성

In [ ]:
date_list = []
while current_date <= end_date :
    if current_date.weekday() < 5:
        #날짜 입력
        wait = WebDriverWait(driver1, 10)
        date_element = wait.until(EC.presence_of_element_located((By.ID,"tmpInqStrDt")
        date_element.clear()
        date_element.send_keys(f"{current_date}")
                                                              
        #조회버튼 클릭                               
        btn = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".btnDefault.bg")))
        btn.click()
        date_list.append(current_date)
        time.sleep(2) 
        df = pd.read_html(StringIO(driver.find_element(By.CSS_SELECTOR, ".tblBasic.leftNone").get_attribute("outerHTML")))[0]                                                         
        df['date'] =  current_date
        new_columes = new_cols                                                         
    current_date += timedelta(days=1)
     

# 테이블 생성하기


In [89]:
driver.find_element(By.CSS_SELECTOR, ".tblBasic.leftNone").get_attribute('outerHTML')

'<table class="tblBasic leftNone" summary="통화,현찰,송금,외화수표파실때,매매기준율,환가료율,미화환산율,사실 때,파실 때,보낼 때,받을 때,환율,Spread,환율,Spread(으)로 이루어진 기간별 기준가격 고시회차에 대한 내용 테이블입니다.">\n\t<caption>기간별 기준가격 고시회차에 대한 내용</caption> \n\t<colgroup>\n\t\t<col style="width:*">\n\t\t<col style="width:8%">\n\t\t<col style="width:7%">\n\t\t<col style="width:8%">\n\t\t<col style="width:7%">\n\t\t<col style="width:8%">\n\t\t<col style="width:7%">\n\t\t<col style="width:8%">\n\t\t<col style="width:8%">\n\t\t<col style="width:8%">\n\t\t<col style="width:8%">\n<!-- \t\t<col style="width:8%" /> -->\n\t</colgroup>\n\t<thead>\n\t\t<tr>\n\t\t\t<th class="leftNone" scope="col" rowspan="3">통화</th>\n\t\t\t<th class="leftLine" scope="col" colspan="4">현찰</th>\n\t\t\t<th class="leftLine" scope="col" colspan="2">송금</th>\n\t\t\n\t\t\t<th class="leftLine" scope="col" rowspan="3">외화<br>수표<br>파실때</th>\n\t\t\t<th class="leftLine" scope="col" rowspan="3">매매<br>기준율</th>\n\t\t\t<th class="leftLine" scope="col" rowspan="3">환가<br>료율</th>\n\t\t\t<th 

In [80]:
from io import StringIO

In [123]:
df = pd.read_html(StringIO(driver.find_element(By.CSS_SELECTOR, ".tblBasic.leftNone").get_attribute("outerHTML")))[0]
df['date'] =  '2024-04-05'
new_columns = new_cols(df)
df.columns = new_columns
df = df[['date','통화','현찰_사실_때_환율','현찰_사실_때_Spread','현찰_파실_때_환율',
         '현찰_파실_때_Spread','송금_보낼_때_보낼_때','송금_받을_때_받을_때',
         '외화_수표_파실때','매매_기준율','환가_료율','미화_환산율']]
df

,date,통화,현찰_사실_때_환율,현찰_사실_때_Spread,현찰_파실_때_환율,현찰_파실_때_Spread,송금_보낼_때_보낼_때,송금_받을_때_받을_때,외화_수표_파실때,매매_기준율,환가_료율,미화_환산율
0,2024-04-05,미국 USD,1504.67,1.75,1452.93,1.75,1493.20,1464.40,1461.88,1478.80,6.14181,1.0000
1,2024-04-05,일본 JPY (100),1035.31,1.75,999.71,1.75,1027.48,1007.54,1006.89,1017.51,2.59591,0.6881
2,2024-04-05,유로 EUR,1662.67,1.99,1597.79,1.99,1646.53,1613.93,1611.98,1630.23,4.32200,1.1024
3,2024-04-05,중국 CNY,210.38,5.00,190.36,5.00,202.37,198.37,0.00,200.37,4.34379,0.1355
4,2024-04-05,홍콩 HKD,194.30,1.97,186.80,1.97,192.45,188.65,188.39,190.55,5.73500,0.1289
5,2024-04-05,태국 THB,44.86,5.00,40.17,6.00,43.15,42.31,42.27,42.73,4.02533,0.0289
6,2024-04-05,대만 TWD,50.72,13.10,40.37,10.00,0.00,0.00,0.00,44.85,3.58766,0.0303
7,2024-04-05,필리핀 PHP,28.36,10.00,23.68,8.20,26.04,25.54,0.00,25.79,7.37000,0.0174
8,2024-04-05,싱가포르 SGD,1118.49,1.99,1074.85,1.99,1107.63,1085.71,1084.43,1096.67,4.75433,0.7416
9,2024-04-05,호주 AUD,906.04,1.97,871.04,1.97,897.42,879.66,878.15,888.54,6.22966,0.6009


# 컬럼 변경하기

In [116]:
df = pd.read_html(StringIO(driver.find_element(By.CSS_SELECTOR, ".tblBasic.leftNone").get_attribute("outerHTML")))[0]
df['date'] =  '2024-04-05'
df.columns

MultiIndex([(       '통화',        '통화',        '통화'),
            (       '현찰',      '사실 때',        '환율'),
            (       '현찰',      '사실 때',    'Spread'),
            (       '현찰',      '파실 때',        '환율'),
            (       '현찰',      '파실 때',    'Spread'),
            (       '송금',      '보낼 때',      '보낼 때'),
            (       '송금',      '받을 때',      '받을 때'),
            ('외화 수표 파실때', '외화 수표 파실때', '외화 수표 파실때'),
            (   '매매 기준율',    '매매 기준율',    '매매 기준율'),
            (    '환가 료율',     '환가 료율',     '환가 료율'),
            (   '미화 환산율',    '미화 환산율',    '미화 환산율'),
            (     'date',          '',          '')],
           )

In [117]:
df.columns[0]

('통화', '통화', '통화')

In [119]:
new_cols =[]
for cols in df.columns:
    if cols[0] == cols[1] == cols[0]:
        new_cols.append(cols[0].replace(" ","_"))
    else:
        new_cols.append(" ".join(cols).strip().replace(" ","_"))
new_cols

['통화',
 '현찰_사실_때_환율',
 '현찰_사실_때_Spread',
 '현찰_파실_때_환율',
 '현찰_파실_때_Spread',
 '송금_보낼_때_보낼_때',
 '송금_받을_때_받을_때',
 '외화_수표_파실때',
 '매매_기준율',
 '환가_료율',
 '미화_환산율',
 'date']

In [121]:
df = pd.read_html(StringIO(driver.find_element(By.CSS_SELECTOR, ".tblBasic.leftNone").get_attribute("outerHTML")))[0]
df['date'] =  '2024-04-05'
new_columns = new_cols(df)
df.columns = new_columns
df

,통화,현찰_사실_때_환율,현찰_사실_때_Spread,현찰_파실_때_환율,현찰_파실_때_Spread,송금_보낼_때_보낼_때,송금_받을_때_받을_때,외화_수표_파실때,매매_기준율,환가_료율,미화_환산율,date
0,미국 USD,1504.67,1.75,1452.93,1.75,1493.20,1464.40,1461.88,1478.80,6.14181,1.0000,2024-04-05
1,일본 JPY (100),1035.31,1.75,999.71,1.75,1027.48,1007.54,1006.89,1017.51,2.59591,0.6881,2024-04-05
2,유로 EUR,1662.67,1.99,1597.79,1.99,1646.53,1613.93,1611.98,1630.23,4.32200,1.1024,2024-04-05
3,중국 CNY,210.38,5.00,190.36,5.00,202.37,198.37,0.00,200.37,4.34379,0.1355,2024-04-05
4,홍콩 HKD,194.30,1.97,186.80,1.97,192.45,188.65,188.39,190.55,5.73500,0.1289,2024-04-05
5,태국 THB,44.86,5.00,40.17,6.00,43.15,42.31,42.27,42.73,4.02533,0.0289,2024-04-05
6,대만 TWD,50.72,13.10,40.37,10.00,0.00,0.00,0.00,44.85,3.58766,0.0303,2024-04-05
7,필리핀 PHP,28.36,10.00,23.68,8.20,26.04,25.54,0.00,25.79,7.37000,0.0174,2024-04-05
8,싱가포르 SGD,1118.49,1.99,1074.85,1.99,1107.63,1085.71,1084.43,1096.67,4.75433,0.7416,2024-04-05
9,호주 AUD,906.04,1.97,871.04,1.97,897.42,879.66,878.15,888.54,6.22966,0.6009,2024-04-05
